<a href="https://colab.research.google.com/github/Dineshkumar-Anbalagan/Sentiment-Analysis-with-RNN/blob/main/Sentiment%20Analysis%20on%20US%20fight%20reviews%20using%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
print("Imported")

Imported


In [9]:
import warnings
warnings.simplefilter('ignore')

In [10]:
data = pd.read_csv("Tweets.csv")
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [11]:
df = data[['airline_sentiment','text']]
df.head(3)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...


In [12]:
df['text'] = df['text'].str.replace('@VirginAmerica', '')
df.head(3)

,airline_sentiment,text
0,neutral,What @dhepburn said.
1,positive,plus you've added commercials to the experien...
2,neutral,I didn't today... Must mean I need to take an...


In [19]:
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '',x))
df['text'].head(3)

0                                   what dhepburn said
1     plus youve added commercials to the experienc...
2     i didnt today must mean i need to take anothe...
Name: text, dtype: object

In [20]:
tokenizer  = Tokenizer(num_words=5000, split = " ")
tokenizer.fit_on_texts(df['text'].values)

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)
X[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   57,  217],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  540,  535, 1122, 2455,    1,    2,  196],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           3,  184,   97,  774,  562,    3,   75,    1,  147,  142,  190],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,   64,  131, 3665,    1, 4598,
        4599,  980,   15,   21, 3092, 3666,   61,   54,   22,  486, 2718],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,   

In [22]:
model = Sequential()
model.add(Embedding(5000, 256, input_length= X.shape[1]))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True, dropout = 0.3, recurrent_dropout=0.2))
model.add(LSTM(256, dropout = 0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [23]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 33, 256)           1280000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 33, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 33, 256)           525312    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 3)                 771       
Total params: 2,331,395
Trainable params: 2,331,395
Non-trainable params: 0
_________________________________________________________________


In [25]:
y = pd.get_dummies(df['airline_sentiment']).values
[print(df['airline_sentiment'][i], y[i]) for i in range(0,5)]

neutral [0 1 0]
positive [0 0 1]
neutral [0 1 0]
negative [1 0 0]
negative [1 0 0]


[None, None, None, None, None]

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [28]:
batch_size = 32
epochs = 8

model.fit(x_train, y_train, epochs=epochs, batch_size = batch_size, verbose = 2)

Epoch 1/8
366/366 - 191s - loss: 0.6418 - accuracy: 0.7370
Epoch 2/8
366/366 - 186s - loss: 0.4353 - accuracy: 0.8351
Epoch 3/8
366/366 - 186s - loss: 0.3451 - accuracy: 0.8716
Epoch 4/8
366/366 - 186s - loss: 0.2779 - accuracy: 0.8980
Epoch 5/8
366/366 - 186s - loss: 0.2333 - accuracy: 0.9141
Epoch 6/8
366/366 - 186s - loss: 0.1902 - accuracy: 0.9310
Epoch 7/8
366/366 - 185s - loss: 0.1566 - accuracy: 0.9424
Epoch 8/8
366/366 - 186s - loss: 0.1327 - accuracy: 0.9521


In [29]:
#model.save('sentiment_analysis.h5')

In [30]:
predictions = model.predict(x_test)
[print(df['text'][i], predictions[i], y_test[i]) for i in range(0, 5)]

 what dhepburn said [9.992342e-01 4.792051e-04 2.866483e-04] [1 0 0]
 plus youve added commercials to the experience tacky [9.9384576e-01 6.1135497e-03 4.0693049e-05] [1 0 0]
 i didnt today must mean i need to take another trip [9.9862051e-01 1.0947401e-03 2.8470604e-04] [1 0 0]
 its really aggressive to blast obnoxious entertainment in your guests faces amp they have little recourse [0.01719879 0.9799256  0.00287572] [1 0 0]
 and its a really big bad thing about it [0.21557327 0.01928731 0.76513946] [1 0 0]


[None, None, None, None, None]

In [33]:
pos_count, neu_count, neg_count = 0, 0, 0
real_pos, real_neu, real_neg = 0, 0, 0
for i, prediction in enumerate(predictions):
  if np.argmax(prediction) == 2:
    pos_count += 1
  elif np.argmax(prediction) == 1:
    neu_count += 1
  else:
    neg_count += 1
  
  if np.argmax(y_test[i]) == 2:
    real_pos += 1
  elif np.argmax(y_test[i]) == 1:
    real_neu += 1
  else:
    real_neg += 1

print('Positive predictions:', pos_count)
print('Neutral predictions:', neu_count)
print('Negative predictions:', neg_count)
print('Real positive:', real_pos)
print('Real Neutral:', real_neu)
print('Real Negative:', real_neg)

Positive predictions: 409
Neutral predictions: 602
Negative predictions: 1917
Real positive: 444
Real Neutral: 614
Real Negative: 1870
